Imports

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import re

1. Загрузите набор данных и изучите его: объем выборки, количество столбцов, характеристики столбцов (признаков), имеются ли пропуски, имеются ли выбросы

In [8]:
# Load the uploaded dataset to examine its structure
data = pd.read_csv('./laptop_price (1).csv', encoding='latin1')

# Display basic information about the dataset
data_info = {
    "shape": data.shape,
    "columns": data.columns.tolist(),
    "data_types": data.dtypes.to_dict(),
    "missing_values": data.isnull().sum().to_dict(),
    "head": data.head()
}
data_info

{'shape': (1303, 13),
 'columns': ['laptop_ID',
  'Company',
  'Product',
  'TypeName',
  'Inches',
  'ScreenResolution',
  'Cpu',
  'Ram',
  'Memory',
  'Gpu',
  'OpSys',
  'Weight',
  'Price_euros'],
 'data_types': {'laptop_ID': dtype('int64'),
  'Company': dtype('O'),
  'Product': dtype('O'),
  'TypeName': dtype('O'),
  'Inches': dtype('float64'),
  'ScreenResolution': dtype('O'),
  'Cpu': dtype('O'),
  'Ram': dtype('O'),
  'Memory': dtype('O'),
  'Gpu': dtype('O'),
  'OpSys': dtype('O'),
  'Weight': dtype('O'),
  'Price_euros': dtype('float64')},
 'missing_values': {'laptop_ID': 0,
  'Company': 0,
  'Product': 0,
  'TypeName': 0,
  'Inches': 0,
  'ScreenResolution': 0,
  'Cpu': 0,
  'Ram': 0,
  'Memory': 0,
  'Gpu': 0,
  'OpSys': 0,
  'Weight': 0,
  'Price_euros': 0},
 'head':    laptop_ID Company      Product   TypeName  Inches  \
 0          1   Apple  MacBook Pro  Ultrabook    13.3   
 1          2   Apple  Macbook Air  Ultrabook    13.3   
 2          3      HP       250 G6   N

2. Подумайте как преобразовать столбцы, чтобы привести все к числовому виду: где стоит категории заменить метками, где преобразовать столбцы. Для этого у всех столбцов типа object посмотрите уникальные значения. 

Столбцы типа данных object ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']

Рекомендации:

- столбец 'Product' можно удалить, в нем  618 уникальных значений

- для столбцов 'RAM'  и 'Weight'  оставить только цифры - объем памяти  и вес, причем в каждом столбце числа должны соответствовать одинаковым единицам Gb и kg

- столбец Memory стоит преобразовать к нескольким столбцам, соответствующим Memory_SSD, Memory_HDD, Memory_Flash_Storage, Memory_Hybrid все объемы памяти перевести в GB и записать в соответствующие столбцы

- решите что делать с остальными столбцами, обосновывая свои решения

In [9]:
# Удаление столбца 'Product'
data = data.drop(columns=['Product'])

# Преобразование 'Ram': удаление текста 'GB' и преобразование к числовому виду
data['Ram'] = data['Ram'].str.replace('GB', '').astype(int)

# Преобразование 'Weight': удаление текста 'kg' и преобразование к числовому виду
data['Weight'] = data['Weight'].str.replace('kg', '').astype(float)

# Преобразование 'Memory' в несколько столбцов
def parse_memory(memory):
    ssd, hdd, flash, hybrid = 0, 0, 0, 0
    if isinstance(memory, str):
        parts = memory.split('+')
        for part in parts:
            size = int(re.search(r'\d+', part).group())
            if 'SSD' in part:
                ssd += size
            elif 'HDD' in part:
                hdd += size
            elif 'Flash' in part:
                flash += size
            elif 'Hybrid' in part:
                hybrid += size
    return pd.Series([ssd, hdd, flash, hybrid])

# Применение функции для разделения столбца 'Memory'
data[['Memory_SSD', 'Memory_HDD', 'Memory_Flash_Storage', 'Memory_Hybrid']] = data['Memory'].apply(parse_memory)
data = data.drop(columns=['Memory'])

# Анализ уникальных значений в текстовых столбцах
text_columns = ['Company', 'TypeName', 'ScreenResolution', 'Cpu', 'Gpu', 'OpSys']
unique_values = {col: data[col].unique() for col in text_columns}

# Вывод уникальных значений для анализа
for col, values in unique_values.items():
    print(f"\n{col} (уникальных значений: {len(values)}):")
    print(values[:10])  # Вывод первых 10 значений


Company (уникальных значений: 19):
['Apple' 'HP' 'Acer' 'Asus' 'Dell' 'Lenovo' 'Chuwi' 'MSI' 'Microsoft'
 'Toshiba']

TypeName (уникальных значений: 6):
['Ultrabook' 'Notebook' 'Netbook' 'Gaming' '2 in 1 Convertible'
 'Workstation']

ScreenResolution (уникальных значений: 40):
['IPS Panel Retina Display 2560x1600' '1440x900' 'Full HD 1920x1080'
 'IPS Panel Retina Display 2880x1800' '1366x768'
 'IPS Panel Full HD 1920x1080' 'IPS Panel Retina Display 2304x1440'
 'IPS Panel Full HD / Touchscreen 1920x1080'
 'Full HD / Touchscreen 1920x1080' 'Touchscreen / Quad HD+ 3200x1800']

Cpu (уникальных значений: 118):
['Intel Core i5 2.3GHz' 'Intel Core i5 1.8GHz'
 'Intel Core i5 7200U 2.5GHz' 'Intel Core i7 2.7GHz'
 'Intel Core i5 3.1GHz' 'AMD A9-Series 9420 3GHz' 'Intel Core i7 2.2GHz'
 'Intel Core i7 8550U 1.8GHz' 'Intel Core i5 8250U 1.6GHz'
 'Intel Core i3 6006U 2GHz']

Gpu (уникальных значений: 110):
['Intel Iris Plus Graphics 640' 'Intel HD Graphics 6000'
 'Intel HD Graphics 620' 'AMD Radeo

3. Столбец laptop_ID  не несет смысловой информации его удалите

In [11]:
# Remove 'laptop_ID' and 'Product' columns
data = data.drop(columns=['laptop_ID'])

4. Сделайте три копии  датасета:  df, df_mm, df_ss. Следующие действия проделайте с каждой из копий:

  Отделите целевой столбец от нецелевых

  Разделите данные на обучающую и тестовую выборки

In [12]:
# Создание копий датасета
df = data.copy()
df_mm = data.copy()
df_ss = data.copy()

# Целевой столбец и признаки
target_column = 'Price_euros'
X = df.drop(columns=[target_column])
y = df[target_column]

# Разделение на обучающую и тестовую выборки (70%/30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Применение Min-Max нормализации к df_mm
scaler_mm = MinMaxScaler()
X_train_mm = scaler_mm.fit_transform(X_train)
X_test_mm = scaler_mm.transform(X_test)

# Применение стандартизации к df_ss
scaler_ss = StandardScaler()
X_train_ss = scaler_ss.fit_transform(X_train)
X_test_ss = scaler_ss.transform(X_test)

# Вывод размеров наборов данных
print("Размеры обучающих и тестовых выборок:")
print(f"Оригинальные данные: X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"Min-Max нормализация: X_train_mm: {X_train_mm.shape}, X_test_mm: {X_test_mm.shape}")
print(f"Стандартизация: X_train_ss: {X_train_ss.shape}, X_test_ss: {X_test_ss.shape}")

NameError: name 'train_test_split' is not defined

5. Первую копию оставьте без изменений. Для второй - примените мин-макс нормализацию, для третьей стандартизацию нецелевых признаков в тестовом и обучающем наборах.

In [ ]:
# Первая копия остается без изменений (df уже разделен на X_train и X_test)

# Вторая копия: Min-Max нормализация
scaler_mm = MinMaxScaler()
X_train_mm = scaler_mm.fit_transform(X_train)
X_test_mm = scaler_mm.transform(X_test)

# Третья копия: стандартизация
scaler_ss = StandardScaler()
X_train_ss = scaler_ss.fit_transform(X_train)
X_test_ss = scaler_ss.transform(X_test)

# Вывод размеров и подтверждение выполнения
print("Преобразования выполнены:")
print(f"Min-Max нормализация: X_train_mm: {X_train_mm.shape}, X_test_mm: {X_test_mm.shape}")
print(f"Стандартизация: X_train_ss: {X_train_ss.shape}, X_test_ss: {X_test_ss.shape}")

6. Примените линейную регрессию, lasso и ridge регрессии с подбором значения гиперпараметра alpha, полиномиальную регрессию со степенями 2 и 3 для каждого из наборов

In [ ]:
# Модели для проверки
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

# Значения alpha для Lasso и Ridge
alphas = [0.01, 0.1, 1, 10, 100]

# Функция для обучения и оценки моделей
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return mean_squared_error(y_test, predictions)

# Результаты для каждой модели
results = {}

# Линейная регрессия
results["LinearRegression"] = {
    "Original": evaluate_model(models["LinearRegression"], X_train, y_train, X_test, y_test),
    "MinMax": evaluate_model(models["LinearRegression"], X_train_mm, y_train, X_test_mm, y_test),
    "Standardized": evaluate_model(models["LinearRegression"], X_train_ss, y_train, X_test_ss, y_test)
}

# Ridge и Lasso регрессия с подбором alpha
for model_name in ["Ridge", "Lasso"]:
    results[model_name] = {}
    for alpha in alphas:
        model = models[model_name].set_params(alpha=alpha)
        results[model_name][f"Original_alpha={alpha}"] = evaluate_model(model, X_train, y_train, X_test, y_test)
        results[model_name][f"MinMax_alpha={alpha}"] = evaluate_model(model, X_train_mm, y_train, X_test_mm, y_test)
        results[model_name][f"Standardized_alpha={alpha}"] = evaluate_model(model, X_train_ss, y_train, X_test_ss, y_test)

# Полиномиальная регрессия степени 2 и 3
for degree in [2, 3]:
    poly = PolynomialFeatures(degree=degree)
    poly_model = Pipeline([
        ("poly_features", poly),
        ("linear_regression", LinearRegression())
    ])
    results[f"PolynomialDegree{degree}"] = {
        "Original": evaluate_model(poly_model, X_train, y_train, X_test, y_test),
        "MinMax": evaluate_model(poly_model, X_train_mm, y_train, X_test_mm, y_test),
        "Standardized": evaluate_model(poly_model, X_train_ss, y_train, X_test_ss, y_test)
    }

# Вывод результатов
for model, res in results.items():
    print(f"\n{model}:")
    for key, value in res.items():
        print(f"  {key}: MSE = {value:.4f}")


7. Сделайте вывод - какая предобработка лучше, какой алгоритм лучше, сравнивая качество на тестовом и обучающем наборах

In [ ]:
# Анализ результатов
# Входные данные: results (содержит ошибки MSE для всех моделей и наборов данных)

# Функция для анализа лучших результатов
def analyze_results(results):
    summary = {}
    for model, res in results.items():
        best_key = min(res, key=res.get)  # Найти ключ с минимальным MSE
        summary[model] = {
            "Best_Set": best_key,
            "Best_MSE": res[best_key]
        }
    return summary

# Получение сводки лучших результатов
best_results = analyze_results(results)

# Вывод лучших моделей и предобработок
print("\nСводка лучших результатов:")
for model, info in best_results.items():
    print(f"{model}: Лучший набор данных - {info['Best_Set']}, MSE = {info['Best_MSE']:.4f}")

# Вывод заключений
print("\nЗаключение:")
print("1. Лучшая предобработка определяется минимальным значением MSE для каждой модели.")
print("2. Полиномиальная регрессия может давать улучшение на сложных зависимостях, но требует внимательной проверки на переобучение.")
print("3. Ridge и Lasso показывают преимущества при наличии регуляризации, особенно для высокоразмерных данных.")
print("4. Min-Max нормализация может быть предпочтительна для моделей, чувствительных к масштабу признаков.")
print("5. Стандартизация чаще лучше работает с регуляризованными методами, такими как Ridge и Lasso.")
